# Comments

First of all, I want to thanks DimitreOliveira for this great script: https://www.kaggle.com/dimitreoliveira/cassava-leaf-disease-tpu-v2-pods-inference.
Also thanks to Chris Deotte for his amazing scripts were I extract some of the functions used in this Kernel.

This script is based on 2 parts, the first one is the training pipeline. The second part is inference.

This was done in colab using 3 TPU-V2. I used 3 TPU-V2 because I realized that my experiments were not deterministic, if I run the same code several times the out of folds give different results and the standard deviation is not small. This is mainly because we have a lot of randomness in our model.

To determine if a change on the model beneffit my CV I average the 3 out of folds, dont know if 3 is enough but is better than 1 haha. This work pretty well and my CV is increasing with new experiments.

Here are the results of the 3 runs

* EXP1_FOLD1_CV: 0.9018
* EXP1_FOLD2_CV: 0.9023
* EXP1_FOLD3_CV: 0.8869
* EXP1_FOLD4_CV: 0.9065
* EXP1_FOLD5_CV: 0.8960
* EXP1_OUT_OF_FOLDS_CV: 0.8987



* EXP2_FOLD1_CV: 0.8993
* EXP2_FOLD2_CV: 0.9023
* EXP2_FOLD3_CV: 0.8958
* EXP2_FOLD4_CV: 0.9056
* EXP2_FOLD5_CV: 0.8969
* EXP2_OUT_OF_FOLDS_CV: 0.9000



* EXP3_FOLD1_CV: 0.8986
* EXP3_FOLD2_CV: 0.9007
* EXP3_FOLD3_CV: 0.8885
* EXP3_FOLD4_CV: 0.9016
* EXP3_FOLD5_CV: 0.8972
* EXP3_OUT_OF_FOLDS_CV: 0.8973

In the inference part I choose the best fold of the 3 experiments, if that is the case we have

* EXP_FOLD1_CV: 0.9018
* EXP_FOLD2_CV: 0.9023
* EXP_FOLD3_CV: 0.8958
* EXP_FOLD4_CV: 0.9065
* EXP_FOLD5_CV: 0.8972
* EXP_MEAN_FOLDS_CV: 0.9007

The folds of the three experiments have the same validation tf records by the way (using same seed for split).

I hope this insights are usefull, out of folds are saved to check blend scores.

# Training Pipeline In Colab

In [ ]:
# !pip install -q efficientnet
# import os
# import re
# import numpy as np
# import pandas as pd
# import random
# import math
# from sklearn import metrics
# from sklearn.model_selection import KFold, StratifiedKFold
# import tensorflow as tf
# import efficientnet.tfkeras as efn
# from tensorflow.keras import backend as K
# import tensorflow_addons as tfa

# try:
#     # TPU detection. No parameters necessary if TPU_NAME environment variable is
#     # set: this is always the case on Kaggle.
#     tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
#     print('Running on TPU ', tpu.master())
# except ValueError:
#     tpu = None

# if tpu:
#     tf.config.experimental_connect_to_cluster(tpu)
#     tf.tpu.experimental.initialize_tpu_system(tpu)
#     strategy = tf.distribute.experimental.TPUStrategy(tpu)
# else:
#     # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
#     strategy = tf.distribute.get_strategy()

# print("REPLICAS: ", strategy.num_replicas_in_sync)

# # For tf.dataset
# AUTO = tf.data.experimental.AUTOTUNE

# # Data access
# GCS_PATH_TRAINING_FILES = 'gs://kds-97ffa76638f61f952661d26d7752f59abdf8f804f1e4794ab6b2e297'
# GCS_PATH_TRAINING_FILES_2019 = 'gs://kds-e707b2c823f5ab0df2845ed8abcbed536d27e188731770fde8a6a95b'

# # Training filenames directories
# TRAINING_FILENAMES = tf.io.gfile.glob(GCS_PATH_TRAINING_FILES + '/*.tfrec')
# TRAINING_FILENAMES_2019 = tf.io.gfile.glob(GCS_PATH_TRAINING_FILES_2019 + '/*.tfrec')

# # Configuration
# EPOCHS = 25
# BATCH_SIZE = 16 * strategy.num_replicas_in_sync
# AUG_BATCH = BATCH_SIZE
# IMAGE_SIZE = [512, 512]
# # Seed
# SEED = 123
# # Learning rate
# LR = 0.0001
# # Test time augmentation rounds
# TTA = 10
# # Verbosity
# VERBOSE = 2
# # Number of classes
# N_CLASSES = 5

# def seed_everything(seed):
#     random.seed(seed)
#     np.random.seed(seed)
#     os.environ['PYTHONHASHSEED'] = str(seed)
#     tf.random.set_seed(seed)

# def cutmix(image, label, PROBABILITY = 1.0):
#     # input image - is a batch of images of size [n,dim,dim,3] not a single image of [dim,dim,3]
#     # output - a batch of images with cutmix applied
#     DIM = IMAGE_SIZE[0]
#     CLASSES = 5
    
#     imgs = []; labs = []
#     for j in range(AUG_BATCH):
#         # DO CUTMIX WITH PROBABILITY DEFINED ABOVE
#         P = tf.cast( tf.random.uniform([],0,1)<=PROBABILITY, tf.int32)
#         # CHOOSE RANDOM IMAGE TO CUTMIX WITH
#         k = tf.cast( tf.random.uniform([],0,AUG_BATCH),tf.int32)
#         # CHOOSE RANDOM LOCATION
#         x = tf.cast( tf.random.uniform([],0,DIM),tf.int32)
#         y = tf.cast( tf.random.uniform([],0,DIM),tf.int32)
#         b = tf.random.uniform([],0,1) # this is beta dist with alpha=1.0
#         WIDTH = tf.cast( DIM * tf.math.sqrt(1-b),tf.int32) * P
#         ya = tf.math.maximum(0,y-WIDTH//2)
#         yb = tf.math.minimum(DIM,y+WIDTH//2)
#         xa = tf.math.maximum(0,x-WIDTH//2)
#         xb = tf.math.minimum(DIM,x+WIDTH//2)
#         # MAKE CUTMIX IMAGE
#         one = image[j,ya:yb,0:xa,:]
#         two = image[k,ya:yb,xa:xb,:]
#         three = image[j,ya:yb,xb:DIM,:]
#         middle = tf.concat([one,two,three],axis=1)
#         img = tf.concat([image[j,0:ya,:,:],middle,image[j,yb:DIM,:,:]],axis=0)
#         imgs.append(img)
#         # MAKE CUTMIX LABEL
#         a = tf.cast(WIDTH*WIDTH/DIM/DIM,tf.float32)
#         if len(label.shape)==1:
#             lab1 = tf.one_hot(label[j],CLASSES)
#             lab2 = tf.one_hot(label[k],CLASSES)
#         else:
#             lab1 = label[j,]
#             lab2 = label[k,]
#         labs.append((1-a)*lab1 + a*lab2)
            
#     # RESHAPE HACK SO TPU COMPILER KNOWS SHAPE OF OUTPUT TENSOR (maybe use Python typing instead?)
#     image2 = tf.reshape(tf.stack(imgs),(AUG_BATCH,DIM,DIM,3))
#     label2 = tf.reshape(tf.stack(labs),(AUG_BATCH,CLASSES))
#     return image2, label2

# def mixup(image, label, PROBABILITY = 1.0):
#     # input image - is a batch of images of size [n,dim,dim,3] not a single image of [dim,dim,3]
#     # output - a batch of images with mixup applied
#     DIM = IMAGE_SIZE[0]
#     CLASSES = 5
    
#     imgs = []; labs = []
#     for j in range(AUG_BATCH):
#         # DO MIXUP WITH PROBABILITY DEFINED ABOVE
#         P = tf.cast( tf.random.uniform([],0,1)<=PROBABILITY, tf.float32)
#         # CHOOSE RANDOM
#         k = tf.cast( tf.random.uniform([],0,AUG_BATCH),tf.int32)
#         a = tf.random.uniform([],0,1)*P # this is beta dist with alpha=1.0
#         # MAKE MIXUP IMAGE
#         img1 = image[j,]
#         img2 = image[k,]
#         imgs.append((1-a)*img1 + a*img2)
#         # MAKE CUTMIX LABEL
#         if len(label.shape)==1:
#             lab1 = tf.one_hot(label[j],CLASSES)
#             lab2 = tf.one_hot(label[k],CLASSES)
#         else:
#             lab1 = label[j,]
#             lab2 = label[k,]
#         labs.append((1-a)*lab1 + a*lab2)
            
#     # RESHAPE HACK SO TPU COMPILER KNOWS SHAPE OF OUTPUT TENSOR (maybe use Python typing instead?)
#     image2 = tf.reshape(tf.stack(imgs),(AUG_BATCH,DIM,DIM,3))
#     label2 = tf.reshape(tf.stack(labs),(AUG_BATCH,CLASSES))
#     return image2, label2

# def data_augment(image, image_name, target):
#     p_spatial = tf.random.uniform([], 0, 1.0, dtype = tf.float32)
#     p_rotate = tf.random.uniform([], 0, 1.0, dtype = tf.float32)
#     p_pixel_1 = tf.random.uniform([], 0, 1.0, dtype = tf.float32)
#     p_pixel_2 = tf.random.uniform([], 0, 1.0, dtype = tf.float32)
#     p_pixel_3 = tf.random.uniform([], 0, 1.0, dtype = tf.float32)
#     p_crop = tf.random.uniform([], 0, 1.0, dtype = tf.float32)
            
#     # Flips
#     image = tf.image.random_flip_left_right(image)
#     image = tf.image.random_flip_up_down(image)
#     if p_spatial > 0.75:
#         image = tf.image.transpose(image)
        
#     # Rotates
#     if p_rotate > 0.75:
#         image = tf.image.rot90(image, k = 3) # rotate 270º
#     elif p_rotate > 0.5:
#         image = tf.image.rot90(image, k = 2) # rotate 180º
#     elif p_rotate > 0.25:
#         image = tf.image.rot90(image, k = 1) # rotate 90º
        
#     # Pixel-level transforms
#     if p_pixel_1 >= 0.4:
#         image = tf.image.random_saturation(image, lower = 0.7, upper = 1.3)
#     if p_pixel_2 >= 0.4:
#         image = tf.image.random_contrast(image, lower = 0.8, upper = 1.2)
#     if p_pixel_3 >= 0.4:
#         image = tf.image.random_brightness(image, max_delta = 0.1)
        
#     # Crops
#     if p_crop > 0.7:
#         if p_crop > 0.9:
#             image = tf.image.central_crop(image, central_fraction = 0.7)
#         elif p_crop > 0.8:
#             image = tf.image.central_crop(image, central_fraction = 0.8)
#         else:
#             image = tf.image.central_crop(image, central_fraction = 0.9)
#     elif p_crop > 0.4:
#         crop_size = tf.random.uniform([], int(IMAGE_SIZE[0] * 0.8), IMAGE_SIZE[0], dtype = tf.int32)
#         image = tf.image.random_crop(image, size = [crop_size, crop_size, 3])

#     image = tf.image.resize(image, size = IMAGE_SIZE)
#     image = tf.reshape(image, [*IMAGE_SIZE, 3])
    
#     return image, image_name, target

# def transform(image, image_name, target):
#     # This Function applies both cutmix and mixup
#     DIM = IMAGE_SIZE[0]
#     CLASSES = 5
#     SWITCH = 0.4
#     CUTMIX_PROB = 0.7
#     MIXUP_PROB = 0.7
#     # For SWITCH PERCENT OF TIME WE DO CUTMIX AND (1-SWITCH) WE DO MIXUP
#     image2, label2 = cutmix(image, target, CUTMIX_PROB)
#     image3, label3 = mixup(image, target, MIXUP_PROB)
#     imgs = []; labs = []
#     for j in range(AUG_BATCH):
#         P = tf.cast( tf.random.uniform([],0,1)<=SWITCH, tf.float32)
#         imgs.append(P*image2[j,]+(1-P)*image3[j,])
#         labs.append(P*label2[j,]+(1-P)*label3[j,])
#     # RESHAPE HACK SO TPU COMPILER KNOWS SHAPE OF OUTPUT TENSOR (maybe use Python typing instead?)
#     image4 = tf.reshape(tf.stack(imgs),(AUG_BATCH,DIM,DIM,3))
#     label4 = tf.reshape(tf.stack(labs),(AUG_BATCH,CLASSES))
#     return image4, image_name, label4

# # Function to decode our images (normalize and reshape)
# def decode_image(image_data):
#     image = tf.image.decode_jpeg(image_data, channels = 3)
#     # Resize image to be aligned with the inference phase
#     image = tf.image.resize(image, IMAGE_SIZE)
#     # convert image to floats in [0, 1] range
#     image = tf.cast(image, tf.float32) / 255.0
#     # explicit size needed for TPU
#     image = tf.reshape(image, [*IMAGE_SIZE, 3])
#     return image

# # This function parse our images and also get the target variable
# def read_labeled_tfrecord(example):
#     LABELED_TFREC_FORMAT = {
#         # tf.string means bytestring
#         "image": tf.io.FixedLenFeature([], tf.string),
#         "image_name": tf.io.FixedLenFeature([], tf.string),
#         # shape [] means single element
#         "target": tf.io.FixedLenFeature([], tf.int64)
#     }
#     example = tf.io.parse_single_example(example, LABELED_TFREC_FORMAT)
#     image = decode_image(example['image'])
#     image_name = example['image_name']
#     target = tf.one_hot(tf.cast(example['target'], tf.int32), N_CLASSES)
#     return image, image_name, target

# def load_dataset(filenames, ordered = False):
#     # Read from TFRecords. For optimal performance, reading from multiple files at once and
#     # Diregarding data order. Order does not matter since we will be shuffling the data anyway
    
#     ignore_order = tf.data.Options()
#     if not ordered:
#         # disable order, increase speed
#         ignore_order.experimental_deterministic = False 
        
#     # Automatically interleaves reads from multiple files
#     dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads = AUTO)
#     # Use data as soon as it streams in, rather than in its original order
#     dataset = dataset.with_options(ignore_order)
#     # Returns a dataset of (image, image_name, label)
#     dataset = dataset.map(read_labeled_tfrecord, num_parallel_calls = AUTO) 
#     return dataset

# def get_training_dataset(filenames, ordered = False):
#     dataset = load_dataset(filenames, ordered = ordered)
#     dataset = dataset.map(data_augment, num_parallel_calls = AUTO)
#     # The training dataset must repeat for several epochs
#     dataset = dataset.repeat()
#     dataset = dataset.batch(AUG_BATCH)
#     dataset = dataset.map(transform, num_parallel_calls = AUTO)
#     dataset = dataset.unbatch()
#     dataset = dataset.shuffle(2048)
#     dataset = dataset.batch(BATCH_SIZE)
#     # Prefetch next batch while training (autotune prefetch buffer size)
#     dataset = dataset.prefetch(AUTO)
#     return dataset

# def get_validation_dataset(filenames, ordered = True):
#     dataset = load_dataset(filenames, ordered = ordered)
#     dataset = dataset.batch(BATCH_SIZE)
#     # Using gpu, not enought memory to use cache
#     # dataset = dataset.cache()
#     # Prefetch next batch while training (autotune prefetch buffer size)
#     dataset = dataset.prefetch(AUTO) 
#     return dataset

# def get_val_tta(filenames, ordered = True):
#     dataset = load_dataset(filenames, ordered = ordered)
#     dataset = dataset.map(data_augment, num_parallel_calls = AUTO)
#     dataset = dataset.repeat()
#     dataset = dataset.batch(BATCH_SIZE)
#     dataset = dataset.prefetch(AUTO)
#     return dataset

# # Function to count how many photos we have in
# def count_data_items(filenames):
#     # The number of data items is written in the name of the .tfrec files, i.e. flowers00-230.tfrec = 230 data items
#     n = [int(re.compile(r"-([0-9]*)\.").search(filename).group(1)) for filename in filenames]
#     return np.sum(n)

# NUM_TRAINING_IMAGES = count_data_items(TRAINING_FILENAMES)
# NUM_TRAINING_IMAGES_2019 = count_data_items(TRAINING_FILENAMES_2019)
# print(f'Dataset: {NUM_TRAINING_IMAGES} 2020 training images')
# print(f'Dataset: {NUM_TRAINING_IMAGES_2019} 2019 training images')

# def get_lr_callback():
#     lr_start   = 0.000001
#     lr_max     = 0.000003 * BATCH_SIZE
#     lr_min     = 0.000001
#     lr_ramp_ep = 1
#     lr_sus_ep  = 0
#     lr_decay   = 0.8
   
#     def lrfn(epoch):
#         if epoch < lr_ramp_ep:
#             lr = (lr_max - lr_start) / lr_ramp_ep * epoch + lr_start   
#         elif epoch < lr_ramp_ep + lr_sus_ep:
#             lr = lr_max    
#         else:
#             lr = (lr_max - lr_min) * lr_decay**(epoch - lr_ramp_ep - lr_sus_ep) + lr_min    
#         return lr

#     lr_callback = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose = False)
#     return lr_callback

# def get_model():
    
#     with strategy.scope():
        
#         inp = tf.keras.layers.Input(shape = (*IMAGE_SIZE, 3))

#         x = efn.EfficientNetB5(weights = 'noisy-student', include_top = False)(inp)
#         x = tf.keras.layers.GlobalAveragePooling2D()(x)
#         x = tf.keras.layers.Dropout(0.2)(x)
#         output = tf.keras.layers.Dense(N_CLASSES, activation = 'softmax')(x)
        
#         model = tf.keras.models.Model(inputs = [inp], outputs = [output])

#         opt = tf.keras.optimizers.Adam(learning_rate = LR)

#         model.compile(
#             optimizer = opt,
#             loss = [tf.keras.losses.CategoricalCrossentropy(label_smoothing = 0.4)],
#             metrics = [tf.keras.metrics.CategoricalAccuracy()]
#         )

#         return model
    
# def train_and_evaluate(folds = 5):
#     oof_image_name = []
#     oof_target = []
#     oof_prediction = np.zeros((NUM_TRAINING_IMAGES, N_CLASSES))
#     previous_number_of_files = 0
#     total_number_of_files = 0
    
#     # Seed everything
#     seed_everything(SEED)
#     kfold = KFold(folds, shuffle = True, random_state = SEED)
#     for fold, (trn_ind, val_ind) in enumerate(kfold.split(TRAINING_FILENAMES)):
#         if tpu:
#             tf.tpu.experimental.initialize_tpu_system(tpu)
#         print('\n')
#         print('-'*50)
#         print(f'Training fold {fold + 1}')
#         train_dataset = get_training_dataset([TRAINING_FILENAMES[x] for x in trn_ind] + TRAINING_FILENAMES_2019, ordered = False)
#         train_dataset = train_dataset.map(lambda image, image_name, target: (image, target))
#         val_dataset = get_validation_dataset([TRAINING_FILENAMES[x] for x in val_ind], ordered = True)
#         val_dataset_ = val_dataset.map(lambda image, image_name, target: (image, target))
#         STEPS_PER_EPOCH = count_data_items([TRAINING_FILENAMES[x] for x in trn_ind] + TRAINING_FILENAMES_2019) // BATCH_SIZE
#         K.clear_session()
#         model = get_model()
#         # Model checkpoint
#         checkpoint = tf.keras.callbacks.ModelCheckpoint(f'EfficientNetB5_EXP1_{fold}_{IMAGE_SIZE[0]}_{SEED}.h5', 
#                                                         monitor = 'val_categorical_accuracy', 
#                                                         verbose = VERBOSE, 
#                                                         save_best_only = True,
#                                                         save_weights_only = True, 
#                                                         mode = 'max')
#         history = model.fit(train_dataset,
#                             steps_per_epoch = STEPS_PER_EPOCH,
#                             epochs = EPOCHS,
#                             callbacks = [checkpoint, get_lr_callback()], 
#                             validation_data = val_dataset_,
#                             verbose = VERBOSE)
        
#         # Load weights from the best epoch
#         model.load_weights(f'EfficientNetB5_EXP1_{fold}_{IMAGE_SIZE[0]}_{SEED}.h5')
        
#         number_of_files = count_data_items([TRAINING_FILENAMES[x] for x in val_ind])
#         # Get validation real target and image name
#         image_name = val_dataset.map(lambda image, image_name, target: image_name).unbatch()
#         target = val_dataset.map(lambda image, image_name, target: target).unbatch()
#         image_name = next(iter(image_name.batch(number_of_files))).numpy().astype('U')
#         target = next(iter(target.batch(number_of_files))).numpy()
#         target = np.argmax(target, axis = -1)
#         oof_image_name.extend(list(image_name))
#         oof_target.extend(list(target))
        
#         # Validation time augmentation
#         steps = TTA * number_of_files / BATCH_SIZE
#         dataset = get_val_tta([TRAINING_FILENAMES[x] for x in val_ind], ordered = True)
#         image = dataset.map(lambda image, image_name, target: image)
#         probabilities = model.predict(image, steps = steps)[: TTA * number_of_files]
#         probabilities = np.mean(probabilities.reshape((number_of_files, TTA, N_CLASSES), order = 'F'), axis = 1)
#         total_number_of_files += number_of_files
#         oof_prediction[previous_number_of_files:total_number_of_files] = probabilities
#         previous_number_of_files += number_of_files
        
#         print('\n')
#         print('-'*50)
#         fold_accuracy_score = metrics.accuracy_score(target, np.argmax(probabilities, axis = -1))
#         print(f'Our fold {fold + 1} accuracy score validation with {TTA} TTA is {fold_accuracy_score}')
        
#     print('\n')
#     print('-'*50)
#     oof_accuracy_score = metrics.accuracy_score(oof_target, np.argmax(oof_prediction, axis = -1))
#     print(f'Our out of folds accuracy score is {oof_accuracy_score}')
    
#     # Save the out of folds predictions
#     print('Saving out of folds to disk...')
#     oof_dataset = pd.DataFrame({'oof_image_name': oof_image_name, 'oof_target': oof_target, 'oof_prediction': list(oof_prediction)})
#     oof_dataset.to_csv(f'EfficientNetB5_EXP1_{IMAGE_SIZE[0]}_{SEED}.csv', index = False)
    
# train_and_evaluate(folds = 5)

# Inference Pipeline

In [ ]:
import os
import re
import numpy as np
import pandas as pd
import random
import math
from sklearn import metrics
from sklearn.model_selection import KFold
import tensorflow as tf
from tensorflow.keras import backend as K
import tensorflow_addons as tfa
from kaggle_datasets import KaggleDatasets
!pip install ../input/cassava-models/Keras_Applications-1.0.8-py3-none-any.whl
!pip install ../input/cassava-models/efficientnet-1.1.0-py3-none-any.whl
import efficientnet.tfkeras as efn
import glob

In [ ]:
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

In [ ]:
# For tf.dataset
AUTO = tf.data.experimental.AUTOTUNE

# # Data access
# GCS_PATH_TRAINING_FILES = KaggleDatasets().get_gcs_path('cassava-leaf-disease-50-tfrecords-center-512x512')
# GCS_PATH_TRAINING_FILES_2019 = KaggleDatasets().get_gcs_path('cassava-leaf-disease-50-tfrecords-external-512x512')

# # Training filenames directories
# TRAINING_FILENAMES = tf.io.gfile.glob(GCS_PATH_TRAINING_FILES + '/*.tfrec')
# TRAINING_FILENAMES_2019 = tf.io.gfile.glob(GCS_PATH_TRAINING_FILES_2019 + '/*.tfrec')

# Configuration
EPOCHS = 20
BATCH_SIZE = 32 * strategy.num_replicas_in_sync
IMAGE_SIZE = [512, 512]
# Seed
SEED = 123
# Learning rate
LR = 0.0001
# Test time augmentation rounds
TTA = 10
# Verbosity
VERBOSE = 2
# Number of classes
N_CLASSES = 5

# Test filenames directory
TEST_FILENAMES = '../input/cassava-leaf-disease-classification/test_images/*.jpg'

In [ ]:
def data_augment(image, image_name):
    p_spatial = tf.random.uniform([], 0, 1.0, dtype = tf.float32)
    p_rotate = tf.random.uniform([], 0, 1.0, dtype = tf.float32)
    p_pixel_1 = tf.random.uniform([], 0, 1.0, dtype = tf.float32)
    p_pixel_2 = tf.random.uniform([], 0, 1.0, dtype = tf.float32)
    p_pixel_3 = tf.random.uniform([], 0, 1.0, dtype = tf.float32)
    p_crop = tf.random.uniform([], 0, 1.0, dtype = tf.float32)
            
    # Flips
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    if p_spatial > 0.75:
        image = tf.image.transpose(image)
        
    # Rotates
    if p_rotate > 0.75:
        image = tf.image.rot90(image, k = 3) # rotate 270º
    elif p_rotate > 0.5:
        image = tf.image.rot90(image, k = 2) # rotate 180º
    elif p_rotate > 0.25:
        image = tf.image.rot90(image, k = 1) # rotate 90º
        
    # Pixel-level transforms
    if p_pixel_1 >= 0.4:
        image = tf.image.random_saturation(image, lower = 0.7, upper = 1.3)
    if p_pixel_2 >= 0.4:
        image = tf.image.random_contrast(image, lower = 0.8, upper = 1.2)
    if p_pixel_3 >= 0.4:
        image = tf.image.random_brightness(image, max_delta = 0.1)
        
    # Crops
    if p_crop > 0.7:
        if p_crop > 0.9:
            image = tf.image.central_crop(image, central_fraction = 0.7)
        elif p_crop > 0.8:
            image = tf.image.central_crop(image, central_fraction = 0.8)
        else:
            image = tf.image.central_crop(image, central_fraction = 0.9)
    elif p_crop > 0.4:
        crop_size = tf.random.uniform([], int(IMAGE_SIZE[0] * 0.8), IMAGE_SIZE[0], dtype = tf.int32)
        image = tf.image.random_crop(image, size = [crop_size, crop_size, 3])

    image = tf.image.resize(image, size = IMAGE_SIZE)
    image = tf.reshape(image, [*IMAGE_SIZE, 3])
    
    return image, image_name

# Function to decode our images (normalize and reshape)
def decode_image(image_data):
    image = tf.image.decode_jpeg(image_data, channels = 3)
    # Resize image to be aligned with the inference phase
    image = tf.image.resize(image, IMAGE_SIZE)
    # convert image to floats in [0, 1] range
    image = tf.cast(image, tf.float32) / 255.0
    # explicit size needed for TPU
    image = tf.reshape(image, [*IMAGE_SIZE, 3])
    return image

def get_image_name(file_path):
    parts = tf.strings.split(file_path, os.path.sep)
    image_name = parts[-1]
    return image_name

def read_image(file_path):
    image_name = get_image_name(file_path)
    image = tf.io.read_file(file_path)
    image = decode_image(image)
    return image, image_name

def get_test_dataset(filenames, tta = False):
    dataset = tf.data.Dataset.list_files(filenames, shuffle = False)
    dataset = dataset.map(read_image, num_parallel_calls = AUTO)
    dataset = dataset.map(data_augment, num_parallel_calls = AUTO)
    # The test dataset must repeat if we want to predict with test time augmentation
    if tta:
        dataset = dataset.repeat() 
    dataset = dataset.batch(BATCH_SIZE)
    # prefetch next batch while predicting (autotune prefetch buffer size)
    dataset = dataset.prefetch(AUTO)
    return dataset


NUM_TESTING_IMAGES = len(os.listdir('../input/cassava-leaf-disease-classification/test_images/'))

In [ ]:
def get_model():
    
    with strategy.scope():
        
        inp = tf.keras.layers.Input(shape = (*IMAGE_SIZE, 3))

        x = efn.EfficientNetB5(weights = None, include_top = False)(inp)
        x = tf.keras.layers.GlobalAveragePooling2D()(x)
        x = tf.keras.layers.Dropout(0.2)(x)
        output = tf.keras.layers.Dense(N_CLASSES, activation = 'softmax')(x)
        
        model = tf.keras.models.Model(inputs = [inp], outputs = [output])

        opt = tf.keras.optimizers.Adam(learning_rate = LR)

        model.compile(
            optimizer = opt,
            loss = [tf.keras.losses.CategoricalCrossentropy(label_smoothing = 0.4)],
            metrics = [tf.keras.metrics.CategoricalAccuracy()]
        )

        return model
    
def inference(model_paths):
    
    # Create a numpy array to store predictions
    prediction = np.zeros((NUM_TESTING_IMAGES, N_CLASSES))
    
    print('Extracting test image names...')
    # Get the test dataset without tta to extract image names
    test_dataset = get_test_dataset(TEST_FILENAMES, tta = False)
    image_name = test_dataset.map(lambda image, image_name: image_name).unbatch()
    image_name = next(iter(image_name.batch(NUM_TESTING_IMAGES))).numpy().astype('U')
    print('Test image names completed...')
    
    for fold, model_path in enumerate(model_paths):
        print('\n')
        print('-'*50)
        print(f'Predicting fold {fold + 1}')
        K.clear_session()
        model = get_model()
        # Load weights of pretrained model
        model.load_weights(model_path)
        
        # Add 1 to the steps because we only have a sample for public inference
        steps = TTA * ((NUM_TESTING_IMAGES / BATCH_SIZE) + 1)
        # Get the test dataset with tta to extract image
        test_dataset = get_test_dataset(TEST_FILENAMES, tta = True)
        image = test_dataset.map(lambda image, image_name: image)
        probabilities = model.predict(image, steps = steps)[: TTA * NUM_TESTING_IMAGES]
        probabilities = np.mean(probabilities.reshape((NUM_TESTING_IMAGES, TTA, N_CLASSES), order = 'F'), axis = 1)
        prediction += probabilities / len(model_paths)
        
    sub = pd.DataFrame({'image_id': image_name, 'label': np.argmax(prediction, axis = -1)})
    sub.to_csv('submission.csv', index = False)
        
    return image_name, prediction, sub
        
# Get pretrained models list for inference
model_paths = glob.glob('../input/cassava-models/*.h5')
image_name, prediction, sub = inference(model_paths)
sub